In [1]:
import duckdb

parquet_file = "drugbank_5.1_2024-01-03.parquet"

conn = duckdb.connect(database=":memory:", read_only=False)
conn.execute(f"CREATE VIEW my_parquet_view AS SELECT * FROM '{parquet_file}'")
schema_result = conn.execute("DESCRIBE my_parquet_view").fetchall()

for column in schema_result:
    print(f"Column Name: {column[0]}, Data Type: {column[1]}")

Column Name: absorption, Data Type: VARCHAR
Column Name: affected_organisms, Data Type: VARCHAR[]
Column Name: ahfs_codes, Data Type: INTEGER[]
Column Name: atc_codes, Data Type: STRUCT(code VARCHAR, "level" STRUCT(code VARCHAR, "text" VARCHAR)[])[]
Column Name: carriers, Data Type: STRUCT(actions VARCHAR[], id VARCHAR, known_action VARCHAR, "name" VARCHAR, organism VARCHAR, polypeptide STRUCT(amino_acid_sequence STRUCT(format VARCHAR, "text" VARCHAR), cellular_location VARCHAR, chromosome_location VARCHAR, external_identifiers STRUCT(identifier VARCHAR, resource VARCHAR)[], gene_name VARCHAR, gene_sequence STRUCT(format VARCHAR, "text" VARCHAR), general_function VARCHAR, go_classifiers STRUCT(category VARCHAR, description VARCHAR)[], id VARCHAR, locus VARCHAR, molecular_weight VARCHAR, "name" VARCHAR, organism STRUCT(ncbi_taxonomy_id VARCHAR, "text" VARCHAR), pfams STRUCT(identifier VARCHAR, "name" VARCHAR)[], signal_regions VARCHAR, source VARCHAR, specific_function VARCHAR, synonyms

In [20]:
import pandas as pd

conn = duckdb.connect(database=":memory:", read_only=False)
conn.execute(f"CREATE VIEW my_parquet_view AS SELECT * FROM '{parquet_file}'")
query_result = conn.query(
    "SELECT drugbank_id[1] AS drugbank_id, description, name, mechanism_of_action, patents FROM my_parquet_view WHERE drugbank_id[1] in ('MESH:C069594', 'DB00118', 'DB00802', 'DB01435', 'MESH:D001647', 'DB09102', 'DB13802', 'MESH:C032881', 'DB00603', 'DB08834', 'DB01586', 'MESH:D014810', 'DB06210', 'DB00175', 'DB00305', 'DB01174', 'MESH:D003520', 'DB00531', 'DB00945', 'DB01005', 'DB01254', 'DB01109')"
)

df = query_result.df()
df_exploded = df.explode('patents')
df_exploded["patents"] = df_exploded["patents"].apply(
    lambda x: x if isinstance(x, dict) else {}
)
df_flattened = df_exploded.join(df_exploded['patents'].apply(pd.Series))
df_flattened = df_flattened.drop('patents', axis=1)
df_flattened = df_flattened.fillna("")
df_flattened = df_flattened.drop_duplicates()

df_flattened.to_excel("selected.xlsx", index=False)

In [11]:
conn = duckdb.connect(database=":memory:", read_only=False)
parquet_file = 'https://drugs.3steps.cn/drugbank_5.1_2024-01-03.parquet'
conn.execute(f"CREATE VIEW my_parquet_view AS SELECT * FROM '{parquet_file}'")
query_result = conn.query(
    "SELECT drugbank_id[1], description, name, mechanism_of_action FROM my_parquet_view LIMIT 10"
)

query_result.df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))